## Start environment

In [ ]:
%%bash
minikube delete -p dc1
minikube start -p dc1
kubectl config use-context dc1
helm repo add hashicorp https://helm.releases.hashicorp.com
helm repo update
rm -rf /tmp/tfe
mkdir /tmp/tfe

## Install PostgresDB

In [ ]:
! kubectl create ns tfe

In [ ]:
%%bash
# Add Helm repository by Bitnami
helm repo add bitnami https://charts.bitnami.com/bitnami

# Update Helm index charts
helm repo update
kubectl apply -f local-pv.yaml
kubectl apply -f pv-claim.yaml
# volumePermissions for EKS
helm install postgresql-dev -f postgres.yaml bitnami/postgresql -n tfe --set volumePermissions.enabled=true

### Create user

In [ ]:
%%bash
helm repo add minio https://charts.min.io/
helm repo update
kubectl create namespace minio
helm install minio minio/minio \
  --namespace minio \
  --set accessKey=myaccesskey \
  --set secretKey=mysecretkey \
  --set replicas=1 \
  --set persistence.enabled=true \
  --set persistence.size=10Gi \
  --set persistence.storageClass=standard \
  --set resources.requests.memory=512Mi \
  --set resources.requests.cpu=250m \
  --set resources.limits.memory=1Gi \
  --set resources.limits.cpu=500m \
  --set mode=standalone

## Install TFE

### CREATE LICENSE

In [ ]:
%%bash
export SERVER="images.releases.hashicorp.com"
export USER="terraform"
export PASSWORD=$(cat ~/tfe_license.txt)
kubectl create secret docker-registry terraform-enterprise --docker-server=$SERVER --docker-username=$USER --docker-password=$PASSWORD  -n tfe

### CREATE CERTIFICATE

In [ ]:
%%bash
export TFE_K8S_NAMESPACE="tfe" \
export TFE_HELM_RELEASE_NAME="tfe" \
export TFE_SERVICE_NAME="terraform-enterprise" \
export K8S_CLUSTER_NAME="cluster.local" \
export WORKDIR=/tmp/tfe

openssl genrsa -out ${WORKDIR}/tfe.key 2048
cat > ${WORKDIR}/tfe-csr.conf <<EOF
[req]
default_bits = 2048
prompt = no
encrypt_key = yes
default_md = sha256
distinguished_name = kubelet_serving
req_extensions = v3_req
[ kubelet_serving ]
O = system:nodes
CN = system:node:*.${TFE_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
[ v3_req ]
basicConstraints = CA:FALSE
keyUsage = nonRepudiation, digitalSignature, keyEncipherment, dataEncipherment
extendedKeyUsage = serverAuth, clientAuth
subjectAltName = @alt_names
[alt_names]
DNS.1 = *.${TFE_SERVICE_NAME}
DNS.2 = *.${TFE_SERVICE_NAME}.${TFE_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
DNS.3 = *.${TFE_HELM_RELEASE_NAME}
DNS.4 = *.${TFE_HELM_RELEASE_NAME}.svc.${K8S_CLUSTER_NAME}
IP.1 = 127.0.0.1
EOF

openssl req -new -key ${WORKDIR}/tfe.key -out ${WORKDIR}/tfe.csr -config ${WORKDIR}/tfe-csr.conf


cat > ${WORKDIR}/csr.yaml <<EOF
apiVersion: certificates.k8s.io/v1
kind: CertificateSigningRequest
metadata:
   name: tfe.svc
spec:
   signerName: kubernetes.io/kubelet-serving
   # signerName: kubernetes.io/kubelet-serving
   expirationSeconds: 8640000
   request: $(cat ${WORKDIR}/tfe.csr|base64|tr -d '\n')
   usages:
   - digital signature
   - key encipherment
   - server auth
EOF

kubectl create -f ${WORKDIR}/csr.yaml
kubectl certificate approve tfe.svc
kubectl get csr tfe.svc
kubectl get csr tfe.svc -o jsonpath='{.status.certificate}' | openssl base64 -d -A -out ${WORKDIR}/tfe.crt
kubectl config view \
--raw \
--minify \
--flatten \
-o jsonpath='{.clusters[].cluster.certificate-authority-data}' \
| base64 -d > ${WORKDIR}/tfe.ca



kubectl create secret generic tfe-tls \
   -n $TFE_K8S_NAMESPACE \
   --from-file=TFE.key=${WORKDIR}/tfe.key \
   --from-file=TFE.crt=${WORKDIR}/tfe.crt \
   --from-file=TFE.ca=${WORKDIR}/tfe.ca


In [ ]:
%env TFE_K8S_NAMESPACE=tfe
%env TFE_HELM_RELEASE_NAME=tfe
%env TFE_SERVICE_NAME=terraform-enterprise 
%env K8S_CLUSTER_NAME=cluster.local
%env WORKDIR=/tmp/tfe

In [39]:
%%bash
cat > ${WORKDIR}/overrides.yaml <<HEREDOC

replicaCount: 1

tls:
  certData: "LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURzekNDQXB1Z0F3SUJBZ0lRVVhKUVBwZUt1czB4RWtNcnVrWStUVEFOQmdrcWhraUc5dzBCQVFzRkFEQVYKTVJNd0VRWURWUVFERXdwdGFXNXBhM1ZpWlVOQk1CNFhEVEkwTVRBeE5ERTFNalF6TUZvWERUSTFNREV5TWpFMQpNalF6TUZvd1JURVZNQk1HQTFVRUNoTU1jM2x6ZEdWdE9tNXZaR1Z6TVN3d0tnWURWUVFERENOemVYTjBaVzA2CmJtOWtaVG9xTG5SbVpTNXpkbU11WTJ4MWMzUmxjaTVzYjJOaGJEQ0NBU0l3RFFZSktvWklodmNOQVFFQkJRQUQKZ2dFUEFEQ0NBUW9DZ2dFQkFLYlpFVlVwQ0RtdUoxR0xxbEI2eDBZRHUrVDQxNG5VMTl4aWRJNDMyay9ZQ1VNMQpTV09ZNjFVSk02YXhzcEFHN1RzeDVtTTNKYU1vV1ZLeHJkOXk1N0RGVE5EblprdkFUSnFObHNFZGI4NlozdXhECkV2cnNZd0dFNDJUSHU0SjFmMlFiTFVZV2lnUDlYaXJueWtOZHJyNDZnYjRML3h3MGNnZFJha09yam1McE0xaWsKSkREL05CbUV6bjlvblZpMDVTTk1Hb1ZvdFVJZzlOVjVlOG1adnBHTkRjTFdxVmxoQWxYUzdNUVVuY3RwZG8wNwppZHQyaURDRlFjMjRVL3NkVlJyMWt2UjBHQ2UxRWJSaFRONEs0dHlTTmhSVC9hSEdXQllDcDY1NzhpTFZZV0dyCkZDUEhOZlU1M2xvYkdlMVM3TzJkMWZGYjhxeEg4cnZjWDVDaWJ5TUNBd0VBQWFPQnpqQ0J5ekFPQmdOVkhROEIKQWY4RUJBTUNCYUF3RXdZRFZSMGxCQXd3Q2dZSUt3WUJCUVVIQXdFd0RBWURWUjBUQVFIL0JBSXdBREFmQmdOVgpIU01FR0RBV2dCUkVoSTIyWVEyaTNjQ0doRTdPVnN0K3lkMHdIekIxQmdOVkhSRUViakJzZ2hZcUxuUmxjbkpoClptOXliUzFsYm5SbGNuQnlhWE5sZ2l3cUxuUmxjbkpoWm05eWJTMWxiblJsY25CeWFYTmxMblJtWlM1emRtTXUKWTJ4MWMzUmxjaTVzYjJOaGJJSUZLaTUwWm1XQ0Z5b3VkR1psTG5OMll5NWpiSFZ6ZEdWeUxteHZZMkZzaHdSLwpBQUFCTUEwR0NTcUdTSWIzRFFFQkN3VUFBNElCQVFBUERmeXVwd2xGREQwWkY2ZUNGdmRwUkVNYnVUaVRyZUFaCmUvRkdTd2lWYVBzTDZzVlRKVlF4b1JOQXZNTUgwTEgyajBNZ05sQ21yMDd4K2xUYXlSNHpVWEhXQ2x5Zm5RQTYKMkVZcHRJNnd6SThKQVdLU3FBUWh3K0ZHZCs5K0krRjUzYmdJUjJpRXpaQzRqNHRpMVI5MUVBejNQUVA5eFhmZQpFcjdRR0VEVUZ6eWYzODlYbFBtb0VqY0xhR050bnhBSXVjZWNwSEwza3VoY0o4V0xTT05CMk1mMXVaN203SnlFCkRXdWhsUndFQmk2ZVBpYVFmQVY1VjB4NmFqYlJmSWpEYkhnSHM5QmhNd2VyblBSelpLY1ZkcTdmc25SSEhhZ3gKVnFMbENXTnlkSEQ3L3ErOWJ4eVBLVEdUbVN6ZnhiK2IyeTAybE5NcXRQZ21vREsySXliaAotLS0tLUVORCBDRVJUSUZJQ0FURS0tLS0tCg=="
  keyData: "LS0tLS1CRUdJTiBSU0EgUFJJVkFURSBLRVktLS0tLQpNSUlFb3dJQkFBS0NBUUVBc3JJYmszTFVjMmZSdHhNWkNFK0dmNDRrbzlNVkcxSkl1czNpRmwrN2IyMkNCclVTCmxvSkc5VkkxWkh4NHZjUCtLeVhHcDA4VllaVFFTRnNYVjd5WFg2UXFpZEdMNU9sVGxRMmdoNHlxZkJVM08wKysKR0pjRnl1ajFGc3R0M3BSWFBUajdCWnRRZCtMT0tsZ0hhZUNmVks5RXdNYUNGSm15elVreG5ZTndSakYvKzdEawpOdm12Q21JK29ET2IwZjhNVFlwdy9nRXFvUW51OXBuckp0dTA0Y3pVa1FZRk1oV0Y0SGxTSzNWbXpsTE9CSVJRCktzem52UzBMbTFGUSt0MVFmVXdIRWlEbTdhazFnZVQrUEZyNjhyUGY0RGx4V2dGaHVNeERhdEhYWEU3cCtVNncKcjdRVlJITlgwbDZpOXQxNjl0UGtXa1dkSThFTUlWK3pTVEFxWndJREFRQUJBb0lCQUZxeTdDdzY3SHBuQjVyVQpwckFCTFZNdzRvZ0VhNElpa2FlL09ESURCUy9aZFZYU01rMFNjdEd4bUVnWXBYQXNPcVhMMXJpMCt6VThRenQyCmdMS1hsbjJFNWRKdCtZT0dxbFhOUk4ySUdFQkN4SFhtVTZaSGdUa3NaTG5kSnpjQ05BVnZGbm9TQ3hYT21MWUYKV0QzMU5Pbld6QjdNb3RteDZYOG8xM1lMY01FdkpWTW00OGM0OGFhZGZheGp4Vlc4cUlKSWZGdURBM3B6K0R1LwpUSGV1bHpndWlQZVlFTlZiT2x6bFloK1FyZWdMa2dTNmUwM3B5R3I4Q0d5dzhkUmhBbW9kd0VNTjdlOThCc3hKCkJYWFZwVWY1bkZmd3lqYjIvYXkwQWhDTlphYUVqcGo5SEhLOHZWU2NYZkErM0MyWTF4bUZFRFRXa2xhUjcwanYKdGJZMDB3RUNnWUVBNE1ZNWRNSEdCWlA4ajlwK0MxRUZPL2pkMjUzM3BxdHlweG9DMWVHWGMxencxZlF5SVd2Tgp6RHo0a0J5Y01nMDNBWkxoRzlSK1MyL0NFa0s0OHlmVytxd2kyK1pjN0FtOVpEUXk5Z2ZZYUtlMkRDS3lRTU45CnVJUTl6VDVQbVUrdTBVbHJ6RURhb3JSbGJkS0plczBWVEVUUERrN2NGNmlscXcwU1JDelBjWmtDZ1lFQXk0VXEKTFBxVlhSOFpyTG5naHBScWZ6RTBDeStHd0s5WWxYQ0t0TTBmc1dYSTVrN0FYbytqVnpDOTl0aG1sZmI0SW9PRQo2YWJzRFkwTGhHdzVrQ1ZoaHR6YmcxU0pteHc5cDU2WTVocnZkaHp5ZHlGS1dBNFIrSG4rQ0UwazI0S2laWXI2CmZodXBiSVRHT3EzNHhVL3FZTXNyRG56a081OEhvZU5zMkJtMSsvOENnWUFQN1h2OHpnNWVMdEppenpUSFVJSnkKY1h3bko5WjR0UVYyMmQwQmt1N01lZldHODdycHExMzE3d3dlQ2lqM1V3Qm0yQ1dqZHpkTDJTdURaNzc3WEFOYgoxLzdRWGs2UFcxRlVYRzZRQ0paSk5Ea3VPdmN1cDZ1cys1SDFqVWpPc0ZyTzhjajF2RkxCM002akxad1lXNWY2Cnc1VjRpaFlHaHlqV0tpTDFPY2VwWVFLQmdDOU8yd2p0anI3QXRmVWltQUJ3aTNqWmRTR01nS1JWbmgvT3MwOW4KeHhBemVzbWVVTFVxYW11Z0Z1aEVVVVJXemtNbmswUEUyMllpL1NSaEM5aG4rQTJqVlJ0b0VUeVBDZjNtcmQvLwpMdDZqeC9ETk9qelJ1SGVyMDlNV0s1VWkvcURmOTI1TSs4c1A2Z3hGMkdmMU5hUWVWZm5lS2lkNXhncHBsU2wrCmdJRDlBb0dCQUtnd3haT05QRjNqL2x2dHdMc2l2QnhPK05uU3kzNU13VlBmbVpNejNjTEJhMU11TUJxM3RnZ3UKdWlPWXVMa2F1OCs1R21OMDRZdTZIUHplUjhEMERLSFVGbXlTY3E5aWlNYXpkdStBbmN6SUVmcDh1N0xPM1lnZgpPQjAzUDBWdzV1VEw1dmhIdW1YejZZODM1cUk3aFc2bjFrSUVFalI2bVpWYVl2YU1wWTZqCi0tLS0tRU5EIFJTQSBQUklWQVRFIEtFWS0tLS0tCg=="
  caCertData: "LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCakNDQWU2Z0F3SUJBZ0lCQVRBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwdGFXNXAKYTNWaVpVTkJNQjRYRFRJeU1URXdPREE1TVRVMU1sb1hEVE15TVRFd05qQTVNVFUxTWxvd0ZURVRNQkVHQTFVRQpBeE1LYldsdWFXdDFZbVZEUVRDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBSnRuCmhabXN2L0JyNHc5djEvUXZGclF2Vk9wNXFqM1lQeXJyYW0ySHlDTnNQMTkvMUhtQ1VlMzYvS3ozUlNiRkFDYzAKdjdGUXJocTVGU2hlZmtBOW5iZVBxOWdkV2NyamdncXByRTFwM2htWjRUQVcybFdkMFczRGh4NjAyWDRuYUFJaApqNTMrbXpvK2tURGRDTjFBTDQvNVJYWWk0a2o5YnBIUGVCZTF2NjRaY0Zvb213NzhENVlYSjNBL1lFZUVqODhwCnZuVjlXMEl5b2ZJWU5HcFFFWnhnTVdyU0kxai9vTUExdVlSN25jM21oeWI5bnk5bmVNMm54TGV5dDI5cDlBQWcKQldkL0cvK1ZjbEVwQzlUOEFCWHhiM3FSYUZHUStLTVpaUm1hcGlXaVFXMWQ2cjY2b0F6cG5kbituTGJ5T1ZJMgpHbytNcCt5MDYxRFZFd2s2eEJrQ0F3RUFBYU5oTUY4d0RnWURWUjBQQVFIL0JBUURBZ0trTUIwR0ExVWRKUVFXCk1CUUdDQ3NHQVFVRkJ3TUNCZ2dyQmdFRkJRY0RBVEFQQmdOVkhSTUJBZjhFQlRBREFRSC9NQjBHQTFVZERnUVcKQkJSRWhJMjJZUTJpM2NDR2hFN09Wc3QreWQwd0h6QU5CZ2txaGtpRzl3MEJBUXNGQUFPQ0FRRUFCT1VJOUE1UwpEWll2YkFwRnNLU1ozaE9TOFNJSXlOK1RHczJLTGFRU2lnVmVvY0VBMnIrcm5ZVEFFRmhmcWV0MnZTejZOYU5vCk1sL3JSZVR2bGlubnh1VHkwQTM0d3pTNGtSMHZMNzVGd2ZMWisxNFZmY3lqZFpCdnd6NlV4bTZPOTViQ1BnMkUKbnMxUUlZQ0V5YXpkWitJTEdZSVpsNWhnRkhoSEZkbUw4Znd2VXY4OWlMTUI2VlR4NWZsVjVDZzBWenpEYlRtYQpUTkZLdlhJcS9oVGxsSDg2eTVqcHlvL0dVTFZjTmNyMVJUcWQ2WWw2MWZ2NHZsajF1VkJhSzQzSW5ueGk0b1NqCjBBdVgwMmc1YlNoMXRaZTdIcS9aeWtjUUF0QnI0WmR4TlBhTnV2WVhITEJGREkxcmdLV25xTFR1Tm5tMk1uRUkKY25QM1BDRXFCRHBMb1E9PQotLS0tLUVORCBDRVJUSUZJQ0FURS0tLS0tCg=="

service:
   type: "ClusterIP" # ClusterIP
   ports:
    - name: "https-443"
      port: 443
      protocol: "TCP"
      targetPort: 8443
   # sessionAffinity: "ClientIP"
image:
 #repository: "images.releases.hashicorp.com"
 name: "hashicorp/terraform-enterprise"
 tag: "v202409-3"
 serviceAccount:
   enabled: true
   #annotations: |
   #   iam.gke.io/gcp-service-account: ${service_account} 
env:
    variables:
        TFE_HOSTNAME: "tfe.tfe.cluster.svc.local"
        TFE_IACT_SUBNETS: "0.0.0.0/0"

        # Database settings.
        TFE_DATABASE_HOST: "postgresql-dev.TFE.svc.cluster.local:5432"
        TFE_DATABASE_NAME: "postgres"
        # TFE_DATABASE_PARAMETERS: <Database extra params e.g "sslmode=require">
        TFE_DATABASE_USER: "postgres"

        TFE_OBJECT_STORAGE_TYPE: s3
        TFE_OBJECT_STORAGE_S3_BUCKET: "test"
        TFE_OBJECT_STORAGE_S3_ENDPOINT: "http://minio.minio.cluster.svc.local:9000"
secrets:
    TFE_DATABASE_PASSWORD: "StrongPassword"
    TFE_OBJECT_STORAGE_S3_ACCESS_KEY_ID: "K0jr5ZxcBcyzjEhmBN0A"
    TFE_OBJECT_STORAGE_S3_SECRET_ACCESS_KEY: "e9Yj1sEGC96lTRGPVEXHDKWvxwm1iIZC53aMdPFh"
    #TFE_LICENSE: "${TFE_LICENSE}"
    TFE_ENCRYPTION_PASSWORD: "Password123!"


HEREDOC



In [ ]:
%%bash
helm install -n $TFE_K8S_NAMESPACE $TFE_HELM_RELEASE_NAME hashicorp/terraform-enterprise -f ${WORKDIR}/overrides.yaml